In [ ]:
##Sobel frame raw.py
##Liam Madden
#1/5/2015

##NOTE: comments that starts with ##GN: are the addition/modification needed
##      to port the script over CPython from MicroPython - Giuseppe Natale

import math

from time import sleep as delay ##GN: changed import n.1: 'as delay' so no need to rewrite code
from pyxi import OVERLAY        ##GN: changed import n.2
from pyxi.video import Frame  ##GN: added import - we need this object in linux because we need contiguous memory allocation
from pyxi.video.hdmi import HDMI
from pyxi.video.vga import VGA

OVERLAY().download_bitstream("audiovideo.bit") ##GN: changed line of code - bitstream download is now slightly different

vga = VGA('out')#define vga as an output port
vga.mode(3) # 1280x720@60Hz
#hdmi = HDMI('in', vga.frame_buffer())#HDMI is in capture mode 'in', not sure what second argument does.
hdmi=HDMI('in')
delay(10) 
hdmi.start()  ##GN: added line of code - now need to explicitly start HDMI as well
delay(10) #takes time for the laptop display to output to HDMI
frame_raw=hdmi.frame_raw()
vga.start()#need to explicetly start VGA monitor
vga.frame_raw(2, frame_raw)
vga.frame_index(2)

width = hdmi.frame_width()
height = hdmi.frame_height()
print('HDMI is capturing a video source of resolution {}x{}'.format(width,height))

def rd_pix(vert, horiz, frame): #read the pixel at vert, horiz in frame and return a list containing rgb
    green=frame[(1920*vert*3) + (3*horiz)]
    blue=frame[(1920*vert*3) + (3*horiz) + 1]
    red=frame[(1920*vert*3) + (3*horiz) + 2]
    rd_pix=[]
    rd_pix.append(red)
    rd_pix.append(green)
    rd_pix.append(blue)
    return rd_pix

def wr_pix(vert, horiz, frame, rgb): #write pixel at vert, horiz with colors in list rgb[]
    frame[(1920*vert*3) + (3*horiz) + 2]=rgb[0] #write red pixel
    frame[(1920*vert*3) + (3*horiz)]=rgb[1] #write green pixel
    frame[(1920*vert*3) + (3*horiz) + 1]=rgb[2] #write blue pixel
    return frame



gray=Frame(width,height).frame  ##GN: changed line of code | allocates a gontiguous buffer. 
sobel=Frame(width,height).frame ##GN: changed line of code | the 'frame' attribute is the raw bytearray underneath this obj

for v in range(0,1024):
    for h in range(0,1280):
        gr = round((0.299*rd_pix(v,h,frame_raw)[0]) +   (0.587*rd_pix(v,h,frame_raw)[1]) + (0.114*rd_pix(v,h,frame_raw)[2]))
        rgb=[]
        rgb.append(gr)
        rgb.append(gr)
        rgb.append(gr)
        gray = wr_pix(v,h,gray,rgb)

vga.frame_raw(0, gray)
vga.frame_index(0)
              
#Compute the Sobel Filter output
#Gx = -1 0 +1     Gy= +1 +2 +1
#     -2 0 +2          0  0  0
#     -1 0 +1         -1 -2 -1

for q in range(1,1023):
    for r in range(1,1279):
        gx= abs(-rd_pix(q+1,r-1,gray)[0] + rd_pix(q+1,r+1,gray)[0] -2*rd_pix(q,r-1,gray)[0] +2*rd_pix(q,r+1,gray)[0] \
        - rd_pix(q-1,r-1,gray)[0] + rd_pix(q-1,r+1, gray)[0])
        gy= abs(rd_pix(q+1,r-1, gray)[0] + 2*rd_pix(q+1,r,gray)[0] + rd_pix(q+1,r+1,gray)[0]  \
        - rd_pix(q-1,r-1,gray)[0]  -2*rd_pix(q-1,r,gray)[0] - rd_pix(q-1,r+1,gray)[0])
        grad=gx + gy
        if grad>255:
            grad=255
        res=[]
        res.append(grad)
        res.append(grad)
        res.append(grad)
        sobel = wr_pix(q,r,sobel,res)

vga.frame_raw(1, sobel)
vga.frame_index(1)

delay(10) #GN: let user see result before exiting the script
del gray #GN: need to release memory
del sobel #GN: need to release memory
del vga #GN: need to release memory
del hdmi #GN: need to release memory
